# [Fe/H] Metallicity

In [14]:
from matplotlib import pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
import pandas as pd

In [15]:
from auriga.snapshot import Snapshot
from auriga.images import figure_setup, set_axs_configuration
from auriga.settings import Settings

In [24]:
figure_setup()

In [10]:
def read_data(simulation: str) -> dict:
    """
    Return a dictionary with the (nan)median values of the [Fe/H]
    abundance ratio for the stars in each component.

    Parameters
    ----------
    simulation : str
        The simulation to consider.

    Returns
    -------
    dict
        A dictionary with the results.
    """
    settings = Settings()

    s = Snapshot(simulation=simulation,
                 loadonlytype=[0, 1, 2, 3, 4, 5])
    s.add_circularity()
    s.add_reference_to_potential()
    s.add_normalized_potential()
    s.tag_particles_by_region(disc_std_circ=1.0,
                              disc_min_circ=0.4,
                              cold_disc_delta_circ=0.25,
                              bulge_max_specific_energy=-0.75)
    s.add_metal_abundance(of="Fe", to="H")

    is_real_star = (s.type == 4) & (s.stellar_formation_time > 0)
    is_main_obj = (s.halo == s.halo_idx) & (s.subhalo == s.subhalo_idx)

    results = {}

    for idx, region in enumerate(settings.components):
        is_region = s.region_tag == idx
        results[region] = np.nanmedian(
            s.metal_abundance["Fe/H"][is_real_star \
                & is_main_obj & is_region]
        )
    results["All"] = np.nanmedian(
            s.metal_abundance["Fe/H"][is_real_star \
                & is_main_obj])

    return results

In [16]:
def accumulate_sample(simulations: list) -> pd.DataFrame:
    """
    Return a Pandas data frame with the (nan)median metallicity
    for the components for each galaxy.

    Parameters
    ----------
    simulations : list
        _description_

    Returns
    -------
    pd.DataFrame
        _description_
    """
    galaxy = []
    data_all = []
    data_halo = []
    data_bulge = []
    data_cdisc = []
    data_wdisc = []
    for simulation in simulations:
        galaxy.append(simulation)
        results = read_data(simulation=simulation)
        data_all.append(results["All"])
        data_halo.append(results["H"])
        data_bulge.append(results["B"])
        data_cdisc.append(results["CD"])
        data_wdisc.append(results["WD"])

    df = pd.DataFrame()
    df["Simulation"] = galaxy
    df["Metallicity_All"] = data_all
    df["Metallicity_H"] = data_halo
    df["Metallicity_B"] = data_bulge
    df["Metallicity_CD"] = data_cdisc
    df["Metallicity_WD"] = data_wdisc

    return df
    

In [20]:
settings = Settings()
originals = [f"au{i}_or_l4_s127" for i in settings.galaxies]
df = accumulate_sample(simulations=originals)

In [69]:
fig = plt.figure(figsize=(2.0, 2.0))
gs = fig.add_gridspec(nrows=1, ncols=1, hspace=0.0, wspace=0.0)
ax = gs.subplots(sharex=True, sharey=True)

ax.grid(True, ls='-', lw=0.25, c='silver')
ax.tick_params(which='both', direction="in")
ax.set_xlim(-0.7, 0.7)
ax.set_ylim(0, 8)
ax.set_xticks([-0.5, 0, 0.5])
ax.set_yticks([0, 2, 4, 6, 8])
ax.set_xlabel('median([Fe/H])')
ax.set_ylabel('PDF')
ax.set_axisbelow(True)

ax.hist(df["Metallicity_All"], bins=15, range=(-0.5, 0.5),
    histtype="stepfilled", color="k", alpha=0.25, lw=0, density=True,
    label="All")
ax.hist(df["Metallicity_H"], bins=15, range=(-0.5, 0.5),
    histtype="step", color="tab:blue", alpha=1, density=True,
    label="Halo")
ax.hist(df["Metallicity_B"], bins=15, range=(-0.5, 0.5),
    histtype="step", color="tab:green", alpha=1, density=True,
    label="Bulge")
ax.hist(df["Metallicity_CD"], bins=15, range=(-0.5, 0.5),
    histtype="step", color="tab:red", alpha=1, density=True,
    label="Cold Disc")
ax.hist(df["Metallicity_WD"], bins=15, range=(-0.5, 0.5),
    histtype="step", color="tab:orange", alpha=1, density=True,
    label="Warm Disc")

ax.legend(loc="upper left", framealpha=0, fontsize=5.0)

fig.savefig("../images/metallicity/FeH_histogram_originals_s127.pdf")
plt.close(fig)